In [1]:
# Download and prepare the data
!cd local && bash ../venv/lib/python3.8/site-packages/fairseq/examples/translation/prepare-iwslt14.sh

fish config running
/home/sondre/.local/bin /home/sondre/TransformerUQ/venv/bin /home/sondre/.vscode-server/bin/cd4ee3b1c348a13bafd8f9ad8060705f6d4b9cba/bin/remote-cli /usr/local/sbin /usr/local/bin /usr/sbin /usr/bin /sbin /bin /usr/games /usr/local/games /usr/lib/wsl/lib /mnt/c/Program Files/WindowsApps/MicrosoftCorporationII.WindowsSubsystemForLinux_2.3.26.0_x64__8wekyb3d8bbwe /mnt/c/WINDOWS/system32 /mnt/c/WINDOWS /mnt/c/WINDOWS/System32/Wbem /mnt/c/WINDOWS/System32/WindowsPowerShell/v1.0/ /mnt/c/WINDOWS/System32/OpenSSH/ /mnt/c/Program Files/PowerShell/7/ /mnt/c/Program Files/Git/cmd /mnt/c/Program Files/Git/mingw64/bin /mnt/c/Program Files/Git/usr/bin /mnt/c/Program Files/GitHub CLI/ /mnt/c/Users/Sondr/.pyenv/pyenv-win/bin /mnt/c/Users/Sondr/.pyenv/pyenv-win/shims /mnt/c/WINDOWS/system32 /mnt/c/WINDOWS /mnt/c/WINDOWS/System32/Wbem /mnt/c/WINDOWS/System32/WindowsPowerShell/v1.0/ /mnt/c/WINDOWS/System32/OpenSSH/ /mnt/c/Program Files/PowerShell/7/ /mnt/c/Program Files/Git/cmd /mnt/c

In [15]:
# Preprocess/binarize the data
!bash -c TEXT=local/iwslt14.tokenized.de-en && \
bash -c fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir local/data-bin/iwslt14.tokenized.de-en \
    --workers 20

fish config running
/home/sondre/.local/bin /home/sondre/TransformerUQ/venv/bin /home/sondre/.vscode-server/bin/cd4ee3b1c348a13bafd8f9ad8060705f6d4b9cba/bin/remote-cli /usr/local/sbin /usr/local/bin /usr/sbin /usr/bin /sbin /bin /usr/games /usr/local/games /usr/lib/wsl/lib /mnt/c/Program Files/WindowsApps/MicrosoftCorporationII.WindowsSubsystemForLinux_2.3.26.0_x64__8wekyb3d8bbwe /mnt/c/WINDOWS/system32 /mnt/c/WINDOWS /mnt/c/WINDOWS/System32/Wbem /mnt/c/WINDOWS/System32/WindowsPowerShell/v1.0/ /mnt/c/WINDOWS/System32/OpenSSH/ /mnt/c/Program Files/PowerShell/7/ /mnt/c/Program Files/Git/cmd /mnt/c/Program Files/Git/mingw64/bin /mnt/c/Program Files/Git/usr/bin /mnt/c/Program Files/GitHub CLI/ /mnt/c/Users/Sondr/.pyenv/pyenv-win/bin /mnt/c/Users/Sondr/.pyenv/pyenv-win/shims /mnt/c/WINDOWS/system32 /mnt/c/WINDOWS /mnt/c/WINDOWS/System32/Wbem /mnt/c/WINDOWS/System32/WindowsPowerShell/v1.0/ /mnt/c/WINDOWS/System32/OpenSSH/ /mnt/c/Program Files/PowerShell/7/ /mnt/c/Program Files/Git/cmd /mnt/c

In [9]:
# train a Transformer translation model over this data
# !CUDA_VISIBLE_DEVICES=0 fairseq-train \
#     local/data-bin/iwslt14.tokenized.de-en \
#     --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
#     --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
#     --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
#     --dropout 0.3 --weight-decay 0.0001 \
#     --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
#     --max-tokens 4096

In [ ]:
# modified to align with hyperparameter from paper
# train a Transformer translation model over this data
!CUDA_VISIBLE_DEVICES=0 fairseq-train \
    local/data-bin/iwslt14.tokenized.de-en \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --max-tokens 4096 \
    --log-interval 10 \
    --no-progress-bar \
    --max-update 500000

2025-01-22 15:44:31 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2025-01-22 15:44:33 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 1, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False

In [2]:
# Evaluate
!fairseq-generate local/data-bin/iwslt14.tokenized.de-en \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe

2025-01-22 15:09:52 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2025-01-22 15:09:53 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': 